<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/main/notebooks/BC5CDR_nameonly_hard_negative_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
import importlib.util

REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"
BRANCH_NAME = "main"

if not os.path.exists(REPO_NAME):
    !git clone -b {BRANCH_NAME} --single-branch {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)


%pip install -q -e "./hypencoder-paper"

os.chdir("hypencoder-paper")

print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

Cloning into 'Hypencoder-Entity-Linking'...
remote: Enumerating objects: 388, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 388 (delta 13), reused 15 (delta 11), pack-reused 367 (from 1)
Receiving objects: 100% (388/388), 849.83 KiB | 16.66 MiB/s, done.
Resolving deltas: 100% (204/204), done.
  Preparing metadata (setup.py) ... done
📍 Working Directory is now: /content/Hypencoder-Entity-Linking/hypencoder-paper/Hypencoder-Entity-Linking/hypencoder-paper
✅ Environment Ready!


In [42]:
from datasets import load_dataset

# there are all "positive" pairs"
dataset = load_dataset("Stevenf232/BC5CDR_MeSH2015_nameonly")

In [43]:
# train_pairs = dataset['train']
# print(train_pairs)

# mention_names = train_pairs['mention']
# entity_names = train_pairs['entity']
# print(mention_names[:3])
# print(entity_names[:3])

### Load the model

In [44]:
# Core Hypencoder model for outputing dense vector representations
from hypencoder_cb.modeling.hypencoder import Hypencoder, HypencoderDualEncoder, TextEncoder
from transformers import AutoTokenizer

model_name = "Stevenf232/SapBERT_freeze_hypencoder"

dual_encoder = HypencoderDualEncoder.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


query_encoder: Hypencoder = dual_encoder.query_encoder
passage_encoder: TextEncoder = dual_encoder.passage_encoder

### Move the model to the GPU

In [45]:
import torch

# Setup the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # This should say 'cuda'

# Move the model to the GPU
passage_encoder.to(device)
query_encoder.to(device)

Using device: cuda


Hypencoder(
  (transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Load datasets and tokenise

In [46]:
# # convert from type "datasets" to python list
# queries = list(mention_names)
# passages = list(entity_names)


# # the output of the tokenizer contains 3 fields:
# # input_ids, token_type_ids, and attention_mask
# # all contain a tensor in the shape (number of queries, max number of tokens)

# query_inputs = tokenizer(queries, return_tensors="pt", padding=True, truncation=True)
# passage_inputs = tokenizer(passages, return_tensors="pt", padding=True, truncation=True)


In [47]:
# print(f"query_inputs:\n{query_inputs}")
# print("\n\n\n")
# print(f"passage_inputs:\n{passage_inputs}")

# Passage Encodings


In [48]:
from tqdm import tqdm
import torch
from torch.amp import autocast

def batch_encode_passages(encoder ,passages):
  batch_size=256
  entity_name_features = []

  num_passages = passages["input_ids"].shape[0]

  with torch.no_grad(): # Disable gradient calculation (saves tons of memory)
    for i in tqdm(range(0, num_passages, batch_size), desc="Extracting features"):

        # extract entity features
        # Autocast does the math in fp16 where possible (default is fp32)
        # this will save memory and increase speed. The loss in precision shouldn't matter much (can check on a small sample if we want)
        with autocast("cuda"):
          features = encoder(
              input_ids=passages["input_ids"][i:i + batch_size].to(device),
              attention_mask=passages["attention_mask"][i:i + batch_size].to(device)
            ).representation

          entity_name_features.append(features.detach().cpu()) # Detach and move to CPU to save VRAM/RAM


  features_tensor = torch.cat(entity_name_features, dim=0)

  return features_tensor

In [49]:
# passage_embeddings = batch_encode_passages(passage_encoder, passage_inputs)

## Now, we create the q-nets.

For each q-net, we feed through it all the passages the calculate the similarity.

But the q-nets are created in batches, and every batch is represented as a single object `NoTorchSequential`. (Check out the `RepeatedDenseBlockConverter` class in q_net.py for more info)

This object expects an input in the shape (N, M, H):

* N = number of queries (mentions)

* M = number of passages (entities)

* H = Hidden dimension (e.g., 768 for BERT)



---



The passage embeddings have the shape (M, H) so we must create an additional dimension of size N.

This will be done like so:
`passages_batch = passages.unsqueeze(0).expand(num_queries, -1, -1)`

* `.unsqueeze()` adds a new dimension (in our case at location 0)

* `.expand()` "expands" that new dimension to be size "num_queries"

* `.expand()` creates a view, so it costs almost 0 memory! (compared to .repeat() which changes the tensor)

# Q-nets take **a lot** of memory.

Instead of creating all of them and then doing the similarity calculation, we will create batches and calculate similarities for just those q-nets, then discard those q-nets and move on to the next batch.

In [50]:
def batch_encode_queries(encoder, queries, passage_embeddings):
  batch_size = 8
  similarity_scores = []

  num_queries = queries["input_ids"].shape[0]

  with torch.no_grad():
    for i in tqdm(range(0, num_queries, batch_size), desc="Creating q-nets and calculating similarity scores"):

        # create q-nets
        with autocast("cuda"):
          q_nets = encoder(
              input_ids=queries["input_ids"][i:i + batch_size].to(device),
              attention_mask=queries["attention_mask"][i:i + batch_size].to(device)
            ).representation


        passages_gpu = passage_embeddings.to(device)

        # Note: we use q_nets.num_queries (our repo's noTorch equivalent of q_nets.shape[0]) instead of batch_size
        # because the total number might not be divisible by batch_size so the last batch might be smaller than the actual batch size
        passages_batch = passages_gpu.unsqueeze(0).expand(q_nets.num_queries, -1, -1)

        # calculate similarity
        batch_scores = q_nets(passages_batch)
        similarity_scores.append(batch_scores.detach().cpu())


  scores_tensor = torch.cat(similarity_scores, dim=0)
  return scores_tensor


In [51]:
# similarity_scores = batch_encode_queries(query_encoder, query_inputs, passage_embeddings)

In [52]:
# Case 1 - comparing a query to its respective passage

# In the simple case where each q_net only takes one passage, we can just
# reshape the passage_embeddings to (N, 1, H).
# passage_embeddings_single = passage_embeddings.unsqueeze(1)
# print(f"passage_embeddings shape: {passage_embeddings_single.shape}")
# giving the nueral network the input of passage_embeddings
# the output provides the relevance score of query 1 against passage 1, query 2 against passage 2, etc...
# scores = q_nets(passage_embeddings_single)
# print(f"scores: {scores}")

In [53]:
# Case 2 - comparing a query to all passages

# The case where each q_net takes multiple passages
# meaning multiple passages are now associated with each of the queries

# this operation creates a 3D tensor which takes too much memory
# passage_embeddings_multi = passage_embeddings.repeat(N, 1).reshape(N, M, H)
# print(f"passage_embeddings shape: {passage_embeddings_multi.shape}")


# unbatched similarity scores for q-nets
# similarity_scores = q_nets(passage_embeddings_multi)
# print(f"similarity_scores shape: {similarity_scores.shape}")
#print(f"similarity_scores: {similarity_scores}")


In [54]:
# similarity_scores

In [55]:
# similarity_scores.shape

## Create a dataset of Hard Negatives based on "Negative Hard Mining"
We take the top "incorrect" item similarities of each query as negatives

In [56]:
"""
Desired format for each line in the JSONL file:
{
  "query": {
    "id": query ID,
    "content": query text,
  },
  "items": [
    {
      "id": passage ID,
      "content": passage text,
      "score": Optional teacher score,
      "type": Sometimes used to specify type of item,
    },
    {
        # another item
    },
  ]
}

Contrastive Loss with Hard Negatives: The positive must be the first item, all following items
will be treated as negative
"""

'\nDesired format for each line in the JSONL file:\n{\n  "query": {\n    "id": query ID,\n    "content": query text,\n  },\n  "items": [\n    {\n      "id": passage ID,\n      "content": passage text,\n      "score": Optional teacher score,\n      "type": Sometimes used to specify type of item,\n    },\n    {\n        # another item\n    },\n  ]\n}\n\nContrastive Loss with Hard Negatives: The positive must be the first item, all following items\nwill be treated as negative\n'

# Perform Hard Negative Mining

In [69]:
import torch
import numpy as np

def write_hardneg_contrastive_jsonl_masked(
    pairs,
    similarity_scores: torch.Tensor,
    output_jsonl_path: str,
    num_negatives: int = 8,
):

    output_jsonl_path = Path(output_jsonl_path)
    output_jsonl_path.parent.mkdir(parents=True, exist_ok=True)

    similarity_scores = similarity_scores.detach().cpu()

    similarity_scores = similarity_scores.squeeze(-1) # flatten last dimenstion
    N, M = similarity_scores.shape

    # Integer Mapping for Speed
    # Convert string IDs to integers. unique_ids[inverse_indices[i]] == pairs["id"][i]
    # We use 'inverse_indices' to check for equality instantly (int vs int)
    # np.unique returns 'inverse' which are the integer indices for each string
    unique_ids, inverse_indices = np.unique(pairs["id"], return_inverse=True)
    id_tensor = torch.from_numpy(inverse_indices) # Shape: (N,)

    # Compare every mention ID (row) against every entity ID (col)
    # unsqueeze(1) makes it (N, 1)
    # unsqueeze(0) makes it (1, M)
    # The mask matches shape of similarity_scores (N, M)
    mask = (id_tensor.unsqueeze(1) == id_tensor.unsqueeze(0))

    # Apply Mask
    # Set any cell where mask is True (same ID) to -infinity.
    # This hides the query itself AND any other rows with the same ID.
    similarity_scores.masked_fill_(mask, -float('inf'))

    # Get Top-K Negatives
    # Since positives are now -inf, topk will only return valid negatives.
    _, top_idxs = torch.topk(similarity_scores, k=num_negatives, dim=1)


    # Extract Negatives with Deduplication
    with output_jsonl_path.open("w", encoding="utf-8") as f:
        for i in range(N):
            q_id = pairs["id"][i]

            # --- Dynamic Fetching Loop ---
            # We need 'num_negatives' unique items.
            # Since duplicates might clog the top of the list, we fetch more than we need.
            # Start with a safe buffer (e.g., 3x what we need + 32).
            k_attempt = (num_negatives * 4) + 32

            valid_neg_indices = []
            seen_entity_ids = set()

            while len(valid_neg_indices) < num_negatives:
                # Cap k at M (total entities)
                if k_attempt > M:
                    k_attempt = M

                # Get top K candidates
                # (Since positives are -inf, these are guaranteed to be negatives)
                _, candidates = torch.topk(similarity_scores[i], k=k_attempt, dim=0, largest=True)

                # Reset collection for this attempt with new k
                valid_neg_indices = []
                seen_entity_ids = set()

                for idx in candidates.tolist():
                    ent_int_id = inverse_indices[idx] # Get the integer ID of this candidate

                    # Deduplicate: Have we seen this Entity ID in this negative list yet?
                    if ent_int_id in seen_entity_ids:
                        continue # Skip duplicate negative

                    # Found a new unique negative
                    seen_entity_ids.add(ent_int_id)
                    valid_neg_indices.append(idx)

                    if len(valid_neg_indices) >= num_negatives:
                        break

                # Check exit conditions
                if len(valid_neg_indices) >= num_negatives:
                    break # Success!
                if k_attempt >= M:
                    break # We searched the entire dataset

                # Not enough unique negatives found? Double the search radius.
                k_attempt *= 2

            # --- Write to File ---
            # Positive
            pos_item = {
                "id": pairs["id"][i],
                "content": pairs["entity"][i],
                "score": None, "type": None
            }

            # Negatives
            neg_items = []
            for idx in valid_neg_indices:
                neg_items.append({
                    "id": pairs["id"][idx],
                    "content": pairs["entity"][idx],
                    "score": None, "type": None
                })

            # Warn if dataset is too small/repetitive
            if len(neg_items) < num_negatives:
                 print(f"Warning: Query {i} found only {len(neg_items)} unique negatives.")

            entry = {
                "query": {"id": q_id, "content": pairs["mention"][i]},
                "items": [pos_item, *neg_items],
            }
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

    print(f"Wrote {N} lines to {output_jsonl_path}")

In [70]:
# generate jsonl for contrastive loss for train/val/test splits

data_splits = ("train", "validation", "test")
seen = set()
splits = [s for s in data_splits if (s in dataset and not (s in seen or seen.add(s)))]

for split in splits:
    print(f"Starting {split} split")
    pairs = dataset[split]

    # build query/passage lists for this split
    queries = list(pairs["mention"])
    passages = list(pairs["entity"])

    # tokenize
    query_inputs = tokenizer(queries, return_tensors="pt", padding=True, truncation=True)
    passage_inputs = tokenizer(passages, return_tensors="pt", padding=True, truncation=True)

    # encode + score
    passage_embeddings = batch_encode_passages(passage_encoder, passage_inputs)
    similarity_scores = batch_encode_queries(query_encoder, query_inputs, passage_embeddings)

    # write jsonl
    write_hardneg_contrastive_jsonl_masked(
        pairs=pairs,
        similarity_scores=similarity_scores,
        output_jsonl_path=f"bc5cdr_{split}_hypencoder_contrastive.jsonl",
        num_negatives=8,
    )

    print("\n\n")

Starting train split


Extracting features: 100%|██████████| 11/11 [00:01<00:00,  8.57it/s]
Creating q-nets and calculating similarity scores: 100%|██████████| 332/332 [00:11<00:00, 27.83it/s]


Wrote 2654 lines to bc5cdr_train_hypencoder_contrastive.jsonl



Starting validation split


Extracting features: 100%|██████████| 10/10 [00:01<00:00,  7.46it/s]
Creating q-nets and calculating similarity scores: 100%|██████████| 320/320 [00:11<00:00, 28.32it/s]


Wrote 2559 lines to bc5cdr_validation_hypencoder_contrastive.jsonl



Starting test split


Extracting features: 100%|██████████| 11/11 [00:01<00:00, 10.73it/s]
Creating q-nets and calculating similarity scores: 100%|██████████| 332/332 [00:11<00:00, 28.36it/s]


Wrote 2656 lines to bc5cdr_test_hypencoder_contrastive.jsonl





## Upload dataset to HuggingFace

In [71]:
from datasets import load_dataset
from pathlib import Path

data_files = {
    "train": "bc5cdr_train_hypencoder_contrastive.jsonl",
    "validation": "bc5cdr_validation_hypencoder_contrastive.jsonl",
    "test": "bc5cdr_test_hypencoder_contrastive.jsonl",
}

# Load as a single DatasetDict
ds = load_dataset("json", data_files=data_files)

# Push to Hub
repo_id = "Stevenf232/BC5CDR_nameonly_hard_negative_mining"
ds.push_to_hub(repo_id, private=False)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########|  195kB /  195kB            

README.md:   0%|          | 0.00/732 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stevenf232/BC5CDR_nameonly_hard_negative_mining/commit/febefb36d0ad50e4ae1eaf4ff0c77cec9ff38b0c', commit_message='Upload dataset', commit_description='', oid='febefb36d0ad50e4ae1eaf4ff0c77cec9ff38b0c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Stevenf232/BC5CDR_nameonly_hard_negative_mining', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Stevenf232/BC5CDR_nameonly_hard_negative_mining'), pr_revision=None, pr_num=None)